# Redes Neuronales Artificales


### Instalar Theano
pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git

### Instalar Tensorflow y Keras
conda install -c conda-forge keras

## Parte 1 - Pre procesado de datos

### Cómo importar las librerías


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


### Importar el data set


In [2]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
dataset = pd.read_csv('/content/drive/My Drive/Curso de Deep Learning de la A a la Z/datasets/Part 1 - Artificial Neural Networks (ANN)/Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values


### Codificar datos categóricos


In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(
    transformers=[
        ("Churn_Modelling",        # Un nombre de la transformación
         OneHotEncoder(categories='auto'), # La clase a la que transformar
         [1]            # Las columnas a transformar.
         )
    ], remainder='passthrough'
)

X = transformer.fit_transform(X)
X = X[:, 1:]


### Dividir el data set en conjunto de entrenamiento y conjunto de testing


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Escalado de variables


In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

## Parte 2 - Construir la RNA




In [ ]:
# Importar Keras y librerías adicionales
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
# Inicializar la RNA
classifier = Sequential()

# Añadir las capas de entrada y primera capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 11))
classifier.add(Dropout(p = 0.1))

# Añadir la segunda capa oculta
classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))
classifier.add(Dropout(p = 0.1))

# Añadir la capa de salida
classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

# Compilar la RNA
classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# Ajustamos la RNA al Conjunto de Entrenamiento
classifier.fit(X_train, y_train,  batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s 151us/step - loss: 0.4843 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4279 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 128us/step - loss: 0.4225 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 131us/step - loss: 0.4187 - acc: 0.8199
Epoch 5/100
8000/8000 [==============================] - 1s 123us/step - loss: 0.4165 - acc: 0.8252
Epoch 6/100
8000/8000 [==============================] - 1s 122us/step - loss: 0.4146 - acc: 0.8299
Epoch 7/100
8000/8000 [==============================] - 1s 119us/step - loss: 0.4131 - acc: 0.8305
Epoch 8/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.4121 - acc: 0.8311
Epoch 9/100
8000/8000 [==============================] - 1s 133us/step - loss: 0.4110 - acc: 0.8322
Epoch 10/100
8000/8000 [==============================] - 1s 126us/step - loss: 0.4098 - acc: 0.8335

## Parte 3 - Evaluar el modelo y calcular predicciones finales


### Predicción de los resultados con el Conjunto de Testing

In [ ]:
y_pred  = classifier.predict(X_test)
y_pred = (y_pred>0.5)

### Elaborar una matriz de confusión



In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1543   52]
 [ 253  152]]


In [ ]:
(cm[0][0]+cm[1][1])/cm.sum()

0.8475

## Parte 4 - Evaluar, mejorar y Ajustar la RNA

### Evaluar la **RNA**

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
def build_classifier():
  # Inicializar la RNA
  classifier = Sequential()

  # Añadir las capas de entrada y primera capa oculta
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                      activation = "relu", input_dim = 11))

  # Añadir la segunda capa oculta
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

  # Añadir la capa de salida
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

  # Compilar la RNA
  classifier.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

  # Devolver el clasificador
  return classifier

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, nb_epoch = 100)
accuracies = cross_val_score(estimator=classifier, X = X_train, y = y_train, cv = 10, n_jobs=-1, verbose = 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   14.7s finished


In [ ]:
accuracies

array([0.78625   , 0.79      , 0.8       , 0.7825    , 0.81625   ,
       0.81      , 0.7875    , 0.79375   , 0.79875   , 0.79499999])

In [ ]:
mean = accuracies.mean()
variance = accuracies.std()
print(mean)
print(variance)

0.7959999969601632
0.010105690620981692


### Mejorar la RNA
#### Regularización de Dropout para evitar el *overfitting*

### Ajustar la *RNA*

In [ ]:
from sklearn.model_selection import GridSearchCV # sklearn.grid_search

In [ ]:
def build_classifier(optimizer):
  # Inicializar la RNA
  classifier = Sequential()

  # Añadir las capas de entrada y primera capa oculta
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  
                      activation = "relu", input_dim = 11))

  # Añadir la segunda capa oculta
  classifier.add(Dense(units = 6, kernel_initializer = "uniform",  activation = "relu"))

  # Añadir la capa de salida
  classifier.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))

  # Compilar la RNA
  classifier.compile(optimizer = optimizer, loss = "binary_crossentropy", metrics = ["accuracy"])

  # Devolver el clasificador
  return classifier

In [ ]:
classifier = KerasClassifier(build_fn = build_classifier)

In [ ]:
parameters = {
    'batch_size' : [25,32],
    'nb_epoch' : [100, 500], 
    'optimizer' : ['adam', 'rmsprop']
}

In [ ]:
grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters, 
                           scoring = 'accuracy', 
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/1
7200/7200 [==============================] - 6s 783us/step - loss: 0.5519 - acc: 0.7956
Epoch 1/1
7200/7200 [==============================] - 6s 784us/step - loss: 0.5476 - acc: 0.7961
Epoch 1/1
7200/7200 [==============================] - 6s 798us/step - loss: 0.5629 - acc: 0.7940
Epoch 1/1
7200/7200 [==============================] - 6s 804us/step - loss: 0.5585 - acc: 0.7972
Epoch 1/1
7200/7200 [==============================] - 6s 813us/step - loss: 0.5824 - acc: 0.7911
Epoch 1/1
7200/7200 [==============================] - 6s 822us/step - loss: 0.5600 - acc: 0.7939
Epoch 1/1
7200/7200 [==============================] - 6s 845us/step - loss: 0.5589 - acc: 0.7949
Epoch 1/1
7200/7200 [==============================] - 6s 855us/step - loss: 0.6001 - acc: 0.7944
Epoch 1/1
7200/7200 [==============================] - 6s 857us/step - loss: 0.5393 - acc: 0.7953
Epoch 1/1
7200/7200 [==============================] - 6s 872us/step - loss: 0.5640 - acc: 0.7947
Epoch 1/1
7200/7200 

KeyboardInterrupt: ignored

In [ ]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 25, 'nb_epoch': 100, 'optimizer': 'adam'}
0.796
